In [18]:
import networkx as nx
from community import community_louvain
import numpy as np
import matplotlib.pyplot as plt
from random import randint,random
import networkx as nx
import matplotlib.pyplot as plt
import torch
import pickle
import scipy.sparse as sparse
#from visualization import RANDOM_STATE
from spectral_cluster_model import  weights_to_graph,cluster_net,delete_isolated_ccs,weights_array_to_cluster_quality
#from visualization import run_spectral_cluster,build_cluster_graph
from collections import Counter
from matplotlib.ticker import MaxNLocator

In [ ]:
dataset="Cora"
file_constraited=dataset+"Convergence"

for epoch in range(0,200,40):
    weights_path0=dataset+"Convergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-"+str(epoch)+".pt"
    weights_array0=torch.load(weights_path0)
    weights_array=[]
    for (i, weight) in enumerate(weights_array0):
        weight = weight[:100,:100].cpu().detach().numpy()
        weights_array.append(weight)
    weights_path=dataset+"Convergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-"+str(epoch)+".pckl"  
    pickle.dump(weights_array,open(weights_path,'wb'))
    
    
    adj_mat=weights_to_graph(weights_array)
    new_weight_array, new_adj_mat=delete_isolated_ccs(weights_array, adj_mat)
   
    num_clusters=6
    assign_labels = 'kmeans'
    eigen_solver = 'amg'
    epsilon=1e-8
    #[ncut_val, clustering_labels]=weights_array_to_cluster_quality(weights_array, adj_mat, num_clusters,eigen_solver, assign_labels, epsilon,is_testing=False)
    #    weights_array_to_cluster_quality() 
    #[labels, metrics]=run_spectral_cluster(weights_path)
    plot_eigenvalues(weights_array)
    #labels=cluster_net(num_clusters, adj_mat, eigen_solver, assign_labels)
    #build_cluster_graph(weights_path,labels,normalize_in_out=True)


In [ ]:
def ToBlockMatrix(weights):
    M,N=weights.shape
    A11=np.zeros((M,M))
    A12=weights
    A21=np.transpose(weights)
    A22=np.zeros((N,N))
    BlockMatrix = np.block([[A11, A12], [A21, A22]])
    return BlockMatrix


In [ ]:
def Adjaencypartition(BlockMatrix):
    BlockMatrix=np.maximum(BlockMatrix,0)
    sp_mat = sparse.coo_matrix(np.abs(BlockMatrix))
    G=nx.from_scipy_sparse_matrix(sp_mat)
    print(G.edges())
    partition=community.best_partition(G) 
    print(collections.Counter(partition.values()))
    """pos=community_layout(G,partition)
    nx.draw(G, pos, node_color=list(partition.values()))
    plt.show()"""

In [ ]:
weight=np.eye(3)
BlockMatrix=ToBlockMatrix(weight)  
BlockMatrix=np.maximum(BlockMatrix,0)
sp_mat = sparse.coo_matrix(np.abs(BlockMatrix))
G=nx.from_scipy_sparse_matrix(sp_mat)
OldParition=community.best_partition(G) 
PartitionClassi=set(OldParition.values())
print("PartitionClassi is",PartitionClassi)
for classi in OldParition.values():
    nodeKnowledge=getKeyList(OldParition,classi)
    print("node is:",nodeKnowledge,classi)


In [2]:
def UpdateWeights(net,OldParition):
    state_dict=torch.load("../Net_state_dict")
    #state_dict = net.state_dict()
    for i,weight in enumerate(state_dict):
        name='layers.{}.weight'.format(i)
        if i==0:
            weight=state_dict[name]
            CorrectWeights(Weight,OldParition)
        else:
            state_dict[name]=weights[i][:-1,:]
        net.load_state_dict(state_dict) 
    return net  

In [3]:
UpdateWeights(1,OldParition)

NameError: name 'OldParition' is not defined

In [4]:
partiontmp

NameError: name 'partiontmp' is not defined

In [5]:
def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    # to install networkx 2.0 compatible version of python-louvain use:
    # pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
    from community import community_louvain

    g = nx.karate_club_graph()
    partition = community_louvain.best_partition(g)
    pos = community_layout(g, partition)

    nx.draw(g, pos, node_color=list(partition.values())); plt.show()
    return

In [6]:
dataset="Cora"
file_constraited=dataset+"Convergence"

for epoch in range(190,220,40):
    weights_path=dataset+"Convergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-"+str(epoch)+".pt"
    print(weights_path)
    weights_array=torch.load(weights_path)
    for weights in weights_array:
        BlockMatrix=ToBlockMatrix(weights.cpu().detach().numpy())
        Adjaencypartition(BlockMatrix)

CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-190.pt


FileNotFoundError: [Errno 2] No such file or directory: 'CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-190.pt'

In [7]:
def GrowConnectivity(G):
    algebraic_connectivity=[]
    for k in range(2):
        algebraic_connectivity.append([])
        for i in range(N):
            G.add_edge(randint(0,N-1),randint(0,N-1))
            algebraic_connectivity[k].append(nx.linalg.algebraic_connectivity(G))

In [8]:
CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-0.pt
Counter({2: 466, 3: 449, 1: 433, 5: 386, 4: 358, 6: 280, 0: 240, 7: 157, 8: 79})
Counter({8: 361, 5: 334, 6: 333, 1: 318, 4: 296, 7: 258, 0: 237, 10: 232, 9: 163, 2: 148, 3: 138})
CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-40.pt
Counter({0: 427, 1: 390, 8: 375, 2: 364, 4: 328, 5: 253, 3: 249, 6: 241, 7: 221})
Counter({3: 443, 1: 349, 2: 340, 5: 332, 8: 274, 4: 267, 0: 255, 9: 229, 7: 219, 6: 110})
CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-80.pt
Counter({6: 434, 5: 432, 1: 399, 4: 385, 2: 308, 8: 253, 3: 240, 7: 229, 0: 168})
Counter({3: 445, 0: 428, 6: 426, 2: 355, 4: 336, 5: 305, 7: 285, 1: 238})
CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-120.pt
Counter({1: 425, 8: 393, 3: 381, 4: 349, 2: 285, 0: 269, 9: 220, 6: 219, 5: 210, 7: 97})
Counter({0: 490, 1: 465, 2: 395, 6: 374, 4: 369, 5: 297, 3: 269, 7: 102, 8: 57})
CoraConvergence/WeightChanges-Cora-GCN-param_512_2_0.99_0.2-monte_0-160.pt
Counter({2: 408, 4: 377, 0: 353, 5: 347, 3: 344, 7: 325, 1: 263, 6: 257, 8: 174})
Counter({6: 421, 4: 385, 0: 375, 5: 338, 8: 319, 3: 317, 1: 253, 2: 218, 7: 192})

SyntaxError: invalid syntax (<ipython-input-8-41b2ea38d002>, line 1)

In [9]:
save_file= open("partitionResults.pkl","wb")
pickle.dump(partition,save_file )
save_file.close()

a_file=open("../partitionResults.pkl",'rb')
OldParition=pickle.load(a_file)

NameError: name 'partition' is not defined

In [10]:
def getKeyList(OldParition,classi):
    list_nodes=[]
    for node in OldParition.keys():
        if OldParition[node]==classi:
            list_nodes.append(node)
    return list_nodes
            

In [11]:
import numpy as np
Weight=np.random.randn(100,100)
print(Weight)

[[-0.6839234   0.44668148  0.54113615 ...  0.43375706 -1.37538939
   0.47379852]
 [ 0.58644499 -0.07685146  0.53444651 ...  0.62825753 -0.05665742
  -0.63545267]
 [-0.23626331 -0.97998268  1.74209448 ...  0.3091554  -0.11039958
   0.53450629]
 ...
 [ 0.55398477  0.62830467  0.7326363  ... -0.14337132 -1.13995021
  -1.01853061]
 [ 1.30696428 -0.57194638  0.96795068 ... -0.28428886 -0.37421041
  -0.98634862]
 [ 1.47076963  0.87406316 -0.15724858 ... -1.48629769  0.77604141
   0.0202894 ]]


In [12]:
def PositiveEdgesInfo(Weight):   
    PositiveEdges=[]
    for i in range(Weight.shape[0]):
        for j in range(Weight.shape[1]): 
            if Weight[i,j]>0:
                PositiveEdges.append((i,j))
    return PositiveEdges
                

In [13]:
def CorrectWeights(Weight,OldParition,PositiveEdges):
    PartitionClassi=set(OldParition.values())
    for classi in PartitionClassi:
        nodeKnowledge=get_key(OldParition,classi)
        for i in range(Weight.shape[0]):
            for j in range(Weight.shape[1]):            
                if (Weight[i,j]<0) and (i in nodeKnowledge) and (j+Weight[1] in nodeKnowledge):
                    Weight[i,j]=0.001
    return Weight

In [14]:
PositiveEdges=PositiveEdgesInfo(Weight)
Weight=CorrectWeights(Weight,OldParition,PositiveEdges)


NameError: name 'OldParition' is not defined

In [15]:
PartitionClassi=set(OldParition.values())
for classi in PartitionClassi:
    print(classi)
    nodes=get_key(OldParition,classi)


NameError: name 'OldParition' is not defined

In [16]:
PartitionClassi=set(OldParition.values())
len(aa)

NameError: name 'OldParition' is not defined

In [17]:
aa.remove(len(aa)-1)
aa

NameError: name 'aa' is not defined

In [93]:
collections.Counter?

In [45]:
for _,valus in c.most_common():
    print(valus)

5
4
3
2
1


In [53]:
collectNodes=[]
for item,value in c.most_common():
    collectNodes.append(OldParition[item].values())
    print(collectNodes)

KeyError: 'a'

In [65]:
OldParition.values()

TypeError: 'dict_values' object does not support indexing

In [68]:
 def get_key (dict, value):
        return [k for k, v in dict.items() if v == value]

In [75]:
for i in range(6):
    nodes=get_key(OldParition,i)
    print(nodes)

[0, 1, 2, 4, 11, 17, 19, 33, 37, 38, 40, 43, 45, 48, 50, 53, 54, 68, 69, 70, 73, 74, 76, 79, 86, 89, 90, 91, 92, 93, 94, 101, 103, 105, 108, 111, 112, 116, 124, 125, 126, 127, 128, 130, 132, 136, 137, 145, 149, 155, 157, 158, 167, 174, 178, 182, 183, 187, 190, 192, 195, 198, 202, 208, 211, 212, 214, 215, 217, 218, 221, 222, 224, 225, 226, 229, 231, 235, 241, 242, 247, 249, 250, 257, 260, 261, 262, 266, 268, 269, 270, 272, 277, 280, 281, 285, 289, 301, 302, 303, 305, 308, 309, 313, 315, 318, 326, 327, 328, 330, 333, 334, 336, 344, 352, 353, 360, 363, 364, 368, 371, 378, 379, 381, 383, 385, 388, 396, 397, 401, 405, 408, 412, 414, 416, 417, 418, 420, 422, 425, 431, 434, 435, 442, 444, 452, 453, 456, 458, 459, 470, 477, 481, 484, 495, 497, 501, 502, 505, 515, 518, 519, 522, 526, 527, 528, 529, 530, 531, 532, 533, 536, 538, 541, 548, 551, 555, 556, 558, 561, 575, 578, 580, 581, 583, 585, 587, 591, 593, 597, 612, 617, 618, 621, 622, 623, 625, 626, 627, 628, 632, 639, 644, 646, 647, 649, 650,

In [ ]:
print(edges)